In [1]:
def parse_vcf(input_file):
    """
    To collect chr and position information of false positives and false negatives from hap.py output vcf
    """
    from collections import defaultdict
    #create dictionary to store results of FN and FP
    output_dict=defaultdict(list)

    with open(input_file, 'r') as infile:
        for line in infile:
            #go to output of VCF
            if line.startswith('#'):
                continue
            #split line into columns
            columns=line.strip().split('\t')
            #only look at valid VCF lines that have 10 or 11 columns
            if len(columns) < 11:
                continue

             #in VCF file, truth is column 9 while QUERY is column 10
            truth_field=columns[9]
            query_field=columns[10]
            
              #split by ':'
            truth_parts=truth_field.split(':')
            query_parts=query_field.split(':')

            #check if FN is in truth field and FP is in query field
            if truth_parts[1] == 'FN':
                output_dict['FN'].append((columns[0],columns[1],truth_parts[4]))

            if query_parts[1] == 'FP':
                output_dict['FP'].append((columns[0], columns[1],query_parts[4]))
    return output_dict
          

In [23]:
collection_falses=parse_vcf("chr20.vcf") #hap.py output vcf file
#separate them to FP and FN
false_positives= list(collection_falses['FP'])
false_negatives=list(collection_falses['FN'])

In [ ]:
def collect_metrics(input_vcf_file, coordinates):
    """
    for each false/true coordinate, go through Clair3 output full alignment vcf file and find variant quality and read depth
    """
    from collections import defaultdict
    collection=defaultdict(list)

    #read VCF into a dictionary
    vcf_dict={}
    with open(input_vcf_file, 'r') as infile:
            for line in infile:
            #go to output of VCF
                if line.startswith('#'):
                    continue
                 #split line into columns
                columns=line.strip().split('\t')
                chromosome=columns[0]
                pos=columns[1]
                sample=columns[9]
                parts=sample.split(':')
                score=parts[1]
                depth=parts[2]
                vcf_dict[(chromosome,pos)]=(score,depth)
    count=0
    found =0
    for variant in coordinates:
        chro=variant[0]
        posi=variant[1]
        if (chro,posi) in vcf_dict:
             collection[(chro,posi)] = (variant[2],vcf_dict[(chro,posi)][0],vcf_dict[(chro,posi)][1])
             found +=1
        else:
             count +=1
    print(f"Missing variants is {count}!")    
    print(f"Found variants is {found}!")
    return collection


In [28]:
metrics = collect_metrics("clair3_work/run4/run4_full_alignment.vcf", false_positives)
with open('false_positives_HG002.txt', 'w') as file:
    for key, value in metrics.items():
        file.write(f"{key}: {value}\n")

Missing variants is 136!
Found variants is 2949!


In [6]:
def collect_TP_coordinates(input_file):
    """
    Collect True Positives chr and position information from hap.py output vcf file
    """
    from collections import defaultdict
    output_dict=defaultdict(list)

    with open(input_file, 'r') as infile:
        for line in infile:
            #go to output of VCF
            if line.startswith('#'):
                continue
            #split line into columns
            columns=line.strip().split('\t')
            #only look at valid VCF lines that have 10 or 11 columns
            if len(columns) < 11:
                continue

             #in VCF file, truth is column 9 
            truth_field=columns[9]
            
              #split by ':'
            truth_parts=truth_field.split(':')
            
            if truth_parts[1] == 'TP':

                output_dict['TP'].append((columns[0],columns[1],truth_parts[4]))

    return output_dict

In [29]:
collection_TP=collect_TP_coordinates("chr20.vcf")

intermid=list(collection_TP.values())
true_TP=intermid[0]
metrics = collect_metrics("clair3_work/run4/run4_full_alignment.vcf", true_TP)
with open('TP_HG002.txt', 'w') as file:
    for key, value in metrics.items():
        file.write(f"{key}: {value}\n")


Missing variants is 14!
Found variants is 44084!


In [8]:
def summary_stats(data_list):
    """
    Calculate basic statistics of genotype quality and read depth for TP and FP calls
    """
    import statistics
    if len(data_list)==0:
        return {"all metrics 0"}
    else:
        data_sorted=sorted(data_list)
        return {
            "mean": round(statistics.mean(data_sorted), 3),
            "median": round(statistics.median(data_sorted), 3),
            "min": round(min(data_sorted), 3),
            "max": round(max(data_sorted), 3),
            "Q3": round(statistics.quantiles(data_sorted, n=4)[2], 3),
            "Q1": round(statistics.quantiles(data_sorted, n=4)[0], 3),
            "IQR": round(statistics.quantiles(data_sorted, n=4)[2], 3) - round(statistics.quantiles(data_sorted, n=4)[0], 3)
        }



In [9]:
def comparison(TP_file,FP_file):
    #create datalist for genotype scores and depth for SNPs and indels for both True positives and false positives
    FP_GT_scores_SNP=list()
    FP_depth_SNP=list()
    FP_GT_scores_INDEL=list()
    FP_depth_INDEL=list()

    TP_GT_scores_SNP=list()
    TP_depth_SNP=list()
    TP_GT_scores_INDEL=list()
    TP_depth_INDEL=list()
    
    with open(TP_file,'r') as infile:
        for line in infile:
            columns=line.split(':')
            variant_info=columns[1].strip().strip("()")
            subparts=variant_info.split(",")
            #strip all troublesome characters and make it clean
            variant_type=subparts[0].strip("() '\"") 
            GTscore=float(subparts[1].strip("() '\""))
            depth=float(subparts[2].strip("() '\"")) 

            if variant_type == "SNP":
                TP_GT_scores_SNP.append(GTscore)
                TP_depth_SNP.append(depth)
            elif variant_type == 'INDEL':
                TP_GT_scores_INDEL.append(GTscore)
                TP_depth_INDEL.append(depth)
    
    with open(FP_file,'r') as infile:
        for line in infile:
            columns=line.split(':')
            variant_info=columns[1].strip().strip("()")
            subparts=variant_info.split(",")
            #strip all troublesome characters and make it clean
            variant_type=subparts[0].strip("() '\"") 
            GTscore=float(subparts[1].strip("() '\""))
            depth=float(subparts[2].strip("() '\"")) 

            if variant_type == "SNP":
                FP_GT_scores_SNP.append(GTscore)
                FP_depth_SNP.append(depth)
            elif variant_type == 'INDEL':
                FP_GT_scores_INDEL.append(GTscore)
                FP_depth_INDEL.append(depth)
                
    # Print summary stats
    print("== TRUE POSITIVE SNP ==")
    print("GT Score:", summary_stats(TP_GT_scores_SNP))
    print("Depth   :", summary_stats(TP_depth_SNP))
    print("\n== TRUE POSITIVE INDEL ==")
    print("GT Score:", summary_stats(TP_GT_scores_INDEL))
    print("Depth   :", summary_stats(TP_depth_INDEL))
    print("\n== FALSE POSITIVE SNP ==")
    print("GT Score:", summary_stats(FP_GT_scores_SNP))
    print("Depth   :", summary_stats(FP_depth_SNP))
    print("\n== FALSE POSITIVE INDEL ==")
    print("GT Score:", summary_stats(FP_GT_scores_INDEL))
    print("Depth   :", summary_stats(FP_depth_INDEL))
        

In [30]:
comparison("TP_HG002.txt","false_positives_HG002.txt")

== TRUE POSITIVE SNP ==
GT Score: {'mean': 24.504, 'median': 25.0, 'min': 0.0, 'max': 110.0, 'Q3': 26.0, 'Q1': 23.0, 'IQR': 3.0}
Depth   : {'mean': 27.687, 'median': 27.0, 'min': 8.0, 'max': 57.0, 'Q3': 31.0, 'Q1': 24.0, 'IQR': 7.0}

== TRUE POSITIVE INDEL ==
GT Score: {'mean': 14.565, 'median': 17.0, 'min': 0.0, 'max': 28.0, 'Q3': 20.0, 'Q1': 9.0, 'IQR': 11.0}
Depth   : {'mean': 27.618, 'median': 27.0, 'min': 10.0, 'max': 55.0, 'Q3': 31.0, 'Q1': 24.0, 'IQR': 7.0}

== FALSE POSITIVE SNP ==
GT Score: {'mean': 13.32, 'median': 14.0, 'min': 0.0, 'max': 24.0, 'Q3': 16.0, 'Q1': 11.0, 'IQR': 5.0}
Depth   : {'mean': 33.878, 'median': 35.0, 'min': 13.0, 'max': 43.0, 'Q3': 37.0, 'Q1': 33.0, 'IQR': 4.0}

== FALSE POSITIVE INDEL ==
GT Score: {'mean': 1.768, 'median': 0.0, 'min': 0.0, 'max': 23.0, 'Q3': 3.0, 'Q1': 0.0, 'IQR': 3.0}
Depth   : {'mean': 26.311, 'median': 26.0, 'min': 10.0, 'max': 54.0, 'Q3': 30.0, 'Q1': 22.0, 'IQR': 8.0}
